# Automated ML

Import Dependencies. 

In [1]:
!pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 25 kB/s s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3


In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AEUKG7W73 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-141541
aml-quickstarts-141541
southcentralus
d4ad7261-832d-46b2-b093-22156001df5b


Create Compute Cluster : Check if a compute cluster exists. Create a new cluster if a cluster doesn't exist. 

In [4]:
amlcompute_cluster_name = "cluster-capstone"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded........................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure. Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

This Heart Failure PRediction dataset is downloaded from Kaggle. Death due to Heart Failure is predicted  using information usch as anaemia, diabetes, high blood pressure, platelets, serum_creatinine, serum_sodium, creatinine_phosporous and ejection_fraction(%bllod leaving the heart at each contraction).

In [5]:
found = False
key = "HeartFailure Data"
description_text = "HeartFailure Data from Kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found: 
    # get the datastore to upload prepared data
    datastore = ws.get_default_datastore()

    # upload the local file from src_dir to the target_path in datastore
    datastore.upload(src_dir='data', target_path='data')

    # create a dataset referencing the cloud location
    dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/heart_failure_clinical_records_dataset.csv'))])
    dataset = dataset.register(workspace=ws, name="HeartFailure Data", description = "HeartFailure Data from Kaggle" )

df = dataset.to_pandas_dataframe()
df.describe()

Uploading an estimated of 1 files
Uploading data/heart_failure_clinical_records_dataset.csv
Uploaded data/heart_failure_clinical_records_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'AUC_weighted'
}

# Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# choose a name for experiment
experiment_name = 'heart-failure-prediction-automl'

experiment=Experiment(ws, experiment_name)

remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion(show_output=True)


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

{'runId': 'AutoML_fea5604c-b61b-4056-a4e4-96cf708b46b0',
 'target': 'cluster-capstone',
 'status': 'Completed',
 'startTimeUtc': '2021-03-29T07:22:44.821614Z',
 'endTimeUtc': '2021-03-29T07:53:53.721993Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cluster-capstone',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-prediction-automl","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-141541","workspace_name":"quick-starts-ws-141541","region":"southcentralus","compute_target":"cluster-capstone","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_c

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)



Run(Experiment: heart-failure-prediction-automl,
Id: AutoML_fea5604c-b61b-4056-a4e4-96cf708b46b0_63,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               silent=None,
                                                                                               subsample=0.9,
                             

In [11]:
model = best_run.register_model(model_path='outputs/model.pkl', model_name='automal_heartfailure_prediction_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

print(model.name, model.id, model.version, sep='\t')

automal_heartfailure_prediction_model	automal_heartfailure_prediction_model:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
env = best_run.get_environment()

script_file = 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file, environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                       memory_gb = 1)

deploy_service_name= 'automl-model-deployment'
service = Model.deploy(ws,deploy_service_name,  [model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

scoring_uri = service.scoring_uri

print("State: ",service.state)
print("\nScoring URI: ", scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-29 07:54:30+00:00 Creating Container Registry if not exists.
2021-03-29 07:54:31+00:00 Registering the environment.
2021-03-29 07:54:31+00:00 Use the existing image.
2021-03-29 07:54:32+00:00 Generating deployment configuration.
2021-03-29 07:54:32+00:00 Submitting deployment to compute.
2021-03-29 07:54:35+00:00 Checking the status of deployment automl-model-deployment..
2021-03-29 07:58:00+00:00 Checking the status of inference endpoint automl-model-deployment.
Succeeded
ACI service creation operation finished, operation "Succeeded"
State:  Healthy

Scoring URI:  http://e024fe18-d111-450c-9650-143a2aca7489.southcentralus.azurecontainer.io/score


TODO: In the cell below, print the logs of the web service and delete the service

In [14]:

# Enable app insights
service.update(enable_app_insights=True)

In [20]:
import requests
import json

# URL for the web service, should be similar to:
#scoring_uri = 'http://0d469fee-33c0-4ce9-9dac-2767356110c9.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
#key = 'zuv0yY9prOPHFssuYSFWDJPhnVMxgJqG'
#two set of data to score, so we get two results back
#age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT

data = {"data": [
    {"age": 75, 
     "anaemia": 0, 
     "creatinine_phosphokinase": 582, 
     "diabetes": 0, 
     "ejection_fraction": 20, 
     "high_blood_pressure": 1, 
     "platelets": 265000, 
     "serum_creatinine": 1.9, 
     "serum_sodium": 130, 
     "sex": 1, 
     "smoking": 0, 
     "time": 4},

    {"age": 55, 
     "anaemia": 0, 
     "creatinine_phosphokinase": 7861, 
     "diabetes": 0, 
     "ejection_fraction": 38, 
     "high_blood_pressure": 0, 
     "platelets": 263358.03, 
     "serum_creatinine": 1.1, 
     "serum_sodium": 136, 
     "sex": 1, 
     "smoking": 0, 
     "time": 6},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0: Death, Case 1: Death.")

{"result": [1, 1]}
Case 0: Death, Case 1: Death.


In [21]:
data = [{"age": 65, 
     "anaemia": 0, 
     "creatinine_phosphokinase": 146, 
     "diabetes": 0, 
     "ejection_fraction": 20, 
     "high_blood_pressure": 0, 
     "platelets": 162000, 
     "serum_creatinine": 1.3, 
     "serum_sodium": 129, 
     "sex": 1, 
     "smoking": 1, 
     "time": 7},
    
    {"age": 55, 
     "anaemia": 0, 
     "creatinine_phosphokinase": 109, 
     "diabetes": 0, 
     "ejection_fraction": 35, 
     "high_blood_pressure": 0, 
     "platelets": 254000, 
     "serum_creatinine": 1.1, 
     "serum_sodium": 139, 
     "sex": 1, 
     "smoking": 1, 
     "time": 60},
      ]
    
print(data)


[{'age': 65, 'anaemia': 0, 'creatinine_phosphokinase': 146, 'diabetes': 0, 'ejection_fraction': 20, 'high_blood_pressure': 0, 'platelets': 162000, 'serum_creatinine': 1.3, 'serum_sodium': 129, 'sex': 1, 'smoking': 1, 'time': 7}, {'age': 55, 'anaemia': 0, 'creatinine_phosphokinase': 109, 'diabetes': 0, 'ejection_fraction': 35, 'high_blood_pressure': 0, 'platelets': 254000, 'serum_creatinine': 1.1, 'serum_sodium': 139, 'sex': 1, 'smoking': 1, 'time': 60}]


In [24]:
# test using service instance
input_data = json.dumps({
    'data': data
})

output = service.run(input_data)
print(output)
print("Case 0: Death, Case 1: No Death.")

{"result": [1, 0]}
Case 0: Death, Case 1: No Death.


In [18]:

logs = service.get_logs()
logs

'2021-03-29T07:57:55,060640800+00:00 - iot-server/run \n2021-03-29T07:57:55,062073000+00:00 - rsyslog/run \n2021-03-29T07:57:55,068672200+00:00 - gunicorn/run \n2021-03-29T07:57:55,083813100+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [25]:
service.delete()